# Re-ranking

Amazon Bedrock은 쿼리 시 검색된 결과의 관련성을 향상시키기 위해 사용할 수 있는 재순위 모델에 대한 액세스를 제공합니다. 
재순위 모델은 청크와 쿼리의 관련성을 계산하고 계산된 점수를 기반으로 결과를 재정렬합니다. 
재순위 모델을 사용하면 쿼리에 답변하는 데 더 적합한 응답을 반환할 수 있습니다.

재순위 모델은 쿼리를 기반으로 관련성 신호를 식별한 다음 해당 신호를 사용하여 문서의 순위를 매기도록 훈련됩니다. 
이로 인해 모델은 더 관련성 있고 정확한 결과를 제공할 수 있습니다.

RAG(Retrieval Augmented Generation) 애플리케이션 구축을 위해 Amazon Bedrock Knowledge Bases를 사용하는 경우, Retrieve 또는 RetrieveAndGenerate 작업을 호출할 때 재순위 모델을 사용하세요. 
재순위 결과는 Amazon Bedrock Knowledge Bases가 결정하는 기본 순위를 재정의합니다.

이 노트북은 Amazon Bedrock Knowledge Bases와 함께 재순위 모델을 사용하는 방법을 보여주며, 
Rerank API를 통해 RAG 애플리케이션의 정확성과 관련성을 더욱 향상시키는 데 도움이 됩니다. 
재순위 모델을 사용하면 더 적지만 더 관련성 있는 결과를 검색할 수 있습니다. 
이러한 결과를 응답 생성에 사용하는 기반 모델에 제공함으로써 비용과 지연 시간을 줄일 수도 있습니다.

Amazon Bedrock Knowledge Bases와 함께 재순위 모델을 구현하고 활용하는 방법을 예제 사용 사례를 통해 살펴보겠습니다.

## 1. Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되었는지 확인하고 Bedrock에 연결해야 합니다.
pip 종속성 오류가 있다면 무시하세요 (라이브러리 설치 중 오류가 보이더라도)

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ragas==0.1.9 --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade boto3

Note: you may need to restart the kernel to use updated packages.


In [4]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [5]:
import boto3
print(boto3.__version__)

1.39.17


In [6]:
import warnings
warnings.filterwarnings('ignore')

이 코드는 설정의 일부이며 다음 용도로 사용됩니다:
- 상위 디렉토리를 파이썬 시스템 경로에 추가
- 나중에 실행하는 데 필요한 utils에서 사용자 정의 모듈(BedrockStructuredKnowledgeBase)을 가져옴

In [7]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [8]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session(region_name = 'us-west-2')
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '461433424192')

In [9]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'reranking-kb'
knowledge_base_description = "Knowledge Base for re-ranking."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

# Define data sources
data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - Create knowledge bases with fixed chunking strategy
csv 형식의 비디오 게임 데이터를 저장하기 위한 [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/)를 생성하는 것부터 시작하겠습니다. 
Knowledge Bases를 사용하면 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() 및 [MongoDB Atlas]()를 포함한 다양한 벡터 데이터베이스와 통합할 수 있습니다. 이 예제에서는 지식 베이스를 Amazon OpenSearch Serverless와 통합합니다. 
이를 위해 지식 베이스와 모든 전제 조건을 생성하는 도우미 클래스 `BedrockKnowledgeBase`를 사용합니다:

1. IAM 역할 및 정책
2. S3 버킷
3. Amazon OpenSearch Serverless 암호화, 네트워크 및 데이터 액세스 정책
4. Amazon OpenSearch Serverless 컬렉션
5. Amazon OpenSearch Serverless 벡터 인덱스
6. 지식 베이스
7. 지식 베이스 데이터 소스

고정 청킹 전략을 사용하여 지식 베이스를 생성합니다.

아래 매개변수 값을 변경하여 다른 청킹 전략을 선택할 수 있습니다:
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [10]:
knowledge_base_metadata = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['reranking-kb-1085248']
buckets_to_check:  ['reranking-kb-1085248']
Creating bucket reranking-kb-1085248
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_1085248) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '314',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 31 Jul 2025 08:52:50 '
                                                 'GMT',
                                         'x-amzn-requestid': '449d2913-3da5-4f16-8e07-654b7b08fa5d'},
                        'HTTPStatusCode': 200,
                        'RequestId': '449d2913-3d

[2025-07-31 08:54:21,335] p2350 {base.py:258} INFO - PUT https://hwdi58tt007x7w6sf1.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-1085248 [status:200 request:0.308s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-1085248',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 31, 8, 55, 21, 462017, tzinfo=tzlocal()),
  'description': 'Knowledge Base for re-ranking.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:461433424192:knowledge-base/CJIZWY7CN1',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'CJIZWY7CN1',
  'name': 'reranking-kb-1085248',
  'roleArn': 'arn:aws:iam::461433424192:role/AmazonBedrockExecutionRoleForKnowledgeBase_1085248',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { '

## 2.1 데이터셋을 Amazon S3에 업로드
이제 지식 베이스를 생성했으니, `Octank financial 10K` 보고서 데이터셋으로 채워보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에서 데이터를 사용할 수 있어야 하며, `StartIngestionJob` 호출을 사용하여 데이터의 변경 사항을 지식 베이스와 동기화할 수 있습니다. 이 예제에서는 우리의 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용할 것입니다.

먼저 dataset 폴더에 있는 메뉴 데이터를 s3에 업로드해보겠습니다.

In [11]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


uploading file ../synthetic_dataset/2024_population.pdf to reranking-kb-1085248


이제 수집 작업을 시작합니다.

In [12]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': '4Y9VLCW5UK',
  'ingestionJobId': 'U3OPYUYPYD',
  'knowledgeBaseId': 'CJIZWY7CN1',
  'startedAt': datetime.datetime(2025, 7, 31, 8, 55, 55, 469680, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 31, 8, 56, 9, 879753, tzinfo=tzlocal())}
........................................

마지막으로 나중에 솔루션을 테스트하기 위해 Knowledge Base ID를 저장합니다.

In [13]:
kb_id = knowledge_base_metadata.get_knowledge_base_id()

'CJIZWY7CN1'


## 3. Evaluate the relevance of query responses with and without Re-ranking (using Ragas)

생성, 평가 및 재순위를 위한 모델 정의

In [14]:
from langchain.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings

bedrock_client = boto3.client('bedrock-runtime')

TEXT_GENERATION_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
EVALUATION_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"

# Reranker model: there are two reranker models available at launch
AMAZON_RERANKER_MODEL_ID = "amazon.rerank-v1:0"
COHERE_RERANKER_MODEL_ID = "cohere.rerank-v3-5:0"


llm_for_evaluation = ChatBedrock(model_id=EVALUATION_MODEL_ID, client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id=EMBEDDING_MODEL_ID, client=bedrock_client)


#### 3.1 Update Knowledge Bases execution role

In [15]:
# Before using autogenerated filters - update the knowledge base execution IAM role with right permissions

iam = boto3.resource('iam')
client = boto3.client('iam')

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

# get the knowledge base IAM role name
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)
role_arn = get_kb_response['knowledgeBase']['roleArn']
role_name = role_arn.split('/')[-1]

# get attached policies
attached_policies = get_attached_policies(role_name)
attached_policies

def update_kb_execution_role(attached_policies, region_name):
    
    for policy in attached_policies:

        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        if 'FoundationModel' in policy_arn:
            print('Updating FoundationModel policy: ',policy_arn)
            policy = iam.Policy(policy_arn)
            version = policy.default_version
            policyJson = version.document
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}')
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{EVALUATION_MODEL_ID}')  
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{AMAZON_RERANKER_MODEL_ID}') 
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{COHERE_RERANKER_MODEL_ID}') 
        
            client.detach_role_policy(RoleName=role_name,
                PolicyArn=policy_arn)
            
            response = client.delete_policy(
                PolicyArn=policy_arn
            )
            print(response)
           
            response = client.create_policy(
            PolicyName= policy_name,
            PolicyDocument=json.dumps(policyJson)
            )
            print(response)
        
        client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

In [16]:
update_kb_execution_role(attached_policies, region)
# time.sleep(30)

arn:aws:iam::461433424192:policy/AmazonBedrockS3PolicyForKnowledgeBase_1085248
arn:aws:iam::461433424192:policy/AmazonBedrockOSSPolicyForKnowledgeBase_1085248
arn:aws:iam::461433424192:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_1085248
arn:aws:iam::461433424192:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_1085248
Updating FoundationModel policy:  arn:aws:iam::461433424192:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_1085248
{'ResponseMetadata': {'RequestId': '66c1e8a1-c70d-4714-8cab-1684e13527ee', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 31 Jul 2025 08:56:52 GMT', 'x-amzn-requestid': '66c1e8a1-c70d-4714-8cab-1684e13527ee', 'content-type': 'text/xml', 'content-length': '204'}, 'RetryAttempts': 0}}
{'Policy': {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_1085248', 'PolicyId': 'ANPAWW34TJFANS6C5PF6D', 'Arn': 'arn:aws:iam::461433424192:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_1085248', 'Path': '/', 'Defau

#### 3.2 Customize retrieve and generate configuraion

In [17]:
def retrieve_and_generate(query, reranker_model=None, kb_id=None, TEXT_GENERATION_MODEL_ID=None, metadata_filters=None):
    
    # Prepare retrieval configuration
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 30 if reranker_model else 3
        }
    }

    if reranker_model:
        retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"] = {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": f'arn:aws:bedrock:{region}::foundation-model/{reranker_model}',
                },
                "numberOfRerankedResults": 3
            }
        }

        if metadata_filters:
            retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"]["bedrockRerankingConfiguration"]["metadataConfiguration"] = {
                                                                "selectionMode" : "SELECTIVE",
                                                                "selectiveModeConfiguration" : {
                                                                    "fieldsToInclude": [{
                                                                        "fieldName": "year",
                                                                    }]
                                                                }
                                                            }
                    

    # Call the retrieve and generate API
    start = time.time()
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}',
                'retrievalConfiguration': retrieval_config,
            },
        }
    )
    time_spent = time.time() - start

    print(f"[Response] : {response['output']['text']}\n")
    print(f"[Invocation time] : {time_spent}\n")

    return response


#### 3.3 Prepare dataset for evaluation

In [22]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness
)

#specify the metrics here
metrics = [
    answer_correctness
]

questions = [
    "2024년 대한민국의 1인 가구의 주요 특징을 연령, 성별, 거주 형태, 그리고 지역적 분포(가장 비율이 높은 시도)를 중심으로 설명해 줘."
]
ground_truths = [
    "2024년 대한민국 1인 가구는 전체 일반 가구의 36.1%를 차지하는 804만 가구에 달하며, 주요 특징은 다음과 같습니다.\
    연령 및 성별 분포: 연령대별로는 20대 이하(17.8%), 60대(17.6%), 30대(17.4%) 순으로 높은 비중을 차지합니다. 성별로 보면, 남성 1인 가구는 30대(21.8%)가, 여성 1인 가구는 60대(19.1%)가 가장 많습니다.\
    지역적 분포: 시도별 1인 가구 비율은 서울이 39.9%로 전국에서 가장 높게 나타났습니다.\
    거주 형태: 1인 가구는 주로 아파트(35.9%)와 다가구단독 주택(20.1%)에 거주하는 것으로 나타났습니다. 이는 전체 일반가구가 아파트에 53.9% 거주하는 것과 비교했을 때, 아파트 거주 비율이 상대적으로 낮은 특징을 보입니다."
]

[2025-07-31 08:58:11,579] p2350 {config.py:112} INFO - TensorFlow version 2.18.0 available.
/opt/conda/lib/python3.12/site-packages/ragas/metrics/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
/opt/conda/lib/python3.12/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a comp

In [23]:
def prepare_eval_dataset(questions, ground_truths, kb_id=None, TEXT_GENERATION_MODEL_ID=None, reranker_model=None, metadata_filters = None):
    answers = []
    contexts = []
    
    for query in questions:
        response = retrieve_and_generate(
            query,
            reranker_model=reranker_model,
            kb_id=kb_id,
            TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
            metadata_filters=metadata_filters
        )
        
        answers.append(response["output"]["text"])
        
        context_group = []
        for citation in response["citations"]:
            context_group.extend([
                ref["content"]["text"]
                for ref in citation["retrievedReferences"]
                if "content" in ref and "text" in ref["content"]
            ])
        contexts.append(context_group)
        time.sleep(15)

    # Create dictionary
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    }

    # Convert dict to dataset
    dataset = Dataset.from_dict(data)
    return dataset


#### 3.4 Evaluate dataset - without re-ranker

In [24]:
without_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=None)

[Response] : 2024년 대한민국의 1인 가구의 주요 특징은 다음과 같습니다:

연령별로는 20대 이하(17.8%), 60대(17.6%), 30대(17.4%) 순으로 1인 가구 비중이 높습니다. 성별로는 남자는 30대(21.8%), 여자는 60대(18.7%)가 가장 많습니다. 지역별로는 서울(39.9%)이 1인 가구 비율이 가장 높고, 울산(31.6%)이 가장 낮습니다.

[Invocation time] : 3.0309441089630127



In [25]:
without_reranker_result = evaluate(
    dataset=without_reranker_dataset,
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

without_reranker_result_df = without_reranker_result.to_pandas()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

[2025-07-31 08:59:03,556] p2350 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-31 08:59:07,973] p2350 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-31 08:59:13,860] p2350 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response


#### 3.5 Evaluate dataset - with re-ranker

In [26]:
with_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=AMAZON_RERANKER_MODEL_ID)

[Response] : 2024년 대한민국의 1인 가구의 주요 특징은 다음과 같습니다:

연령별로는 20대 이하(17.8%), 60대(17.6%), 30대(17.4%) 순으로 비중이 높습니다. 성별로는 남자는 30대(21.8%), 여자는 60대(19.1%)의 비중이 가장 높습니다. 거주 형태별로는 1인 가구의 35.9%가 아파트에, 20.1%가 다가구단독 주택에 거주하고 있습니다.

지역별로는 서울(39.9%)이 1인 가구 비율이 가장 높고, 울산(31.6%)이 가장 낮습니다.

[Invocation time] : 3.9023971557617188



In [27]:
with_reranker_result = evaluate(
dataset=with_reranker_dataset,
metrics=metrics,
llm=llm_for_evaluation,
embeddings=bedrock_embeddings,
)

with_reranker_result_df = with_reranker_result.to_pandas()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

[2025-07-31 09:00:08,876] p2350 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-31 09:00:13,749] p2350 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-31 09:00:18,351] p2350 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response


#### 3.5 Prepare Comparison data frame

In [28]:
import pandas as pd

# Create the side-by-side DataFrame
comparison_df = pd.DataFrame({
    'question': without_reranker_result_df['question'],
    'without_reranker_answer': without_reranker_result_df['answer'],
    'with_reranker_answer': with_reranker_result_df['answer'],
    
    'without_reranker_answer_correctness': without_reranker_result_df['answer_correctness'],
    'with_reranker_answer_correctness': with_reranker_result_df['answer_correctness'],
    })

In [29]:
pd.options.display.max_colwidth = 1000
comparison_df

,question,without_reranker_answer,with_reranker_answer,without_reranker_answer_correctness,with_reranker_answer_correctness
0,"2024년 대한민국의 1인 가구의 주요 특징을 연령, 성별, 거주 형태, 그리고 지역적 분포(가장 비율이 높은 시도)를 중심으로 설명해 줘.","2024년 대한민국의 1인 가구의 주요 특징은 다음과 같습니다:\n\n연령별로는 20대 이하(17.8%), 60대(17.6%), 30대(17.4%) 순으로 1인 가구 비중이 높습니다. 성별로는 남자는 30대(21.8%), 여자는 60대(18.7%)가 가장 많습니다. 지역별로는 서울(39.9%)이 1인 가구 비율이 가장 높고, 울산(31.6%)이 가장 낮습니다.","2024년 대한민국의 1인 가구의 주요 특징은 다음과 같습니다:\n\n연령별로는 20대 이하(17.8%), 60대(17.6%), 30대(17.4%) 순으로 비중이 높습니다. 성별로는 남자는 30대(21.8%), 여자는 60대(19.1%)의 비중이 가장 높습니다. 거주 형태별로는 1인 가구의 35.9%가 아파트에, 20.1%가 다가구단독 주택에 거주하고 있습니다.\n\n지역별로는 서울(39.9%)이 1인 가구 비율이 가장 높고, 울산(31.6%)이 가장 낮습니다.",0.790797,0.865416


In [30]:
# Calculate average correctness
without_reranker_avg_correctness = without_reranker_result_df['answer_correctness'].mean()
with_reranker_avg_correctness = with_reranker_result_df['answer_correctness'].mean()

print(f"\nAverage Correctness without Reranker: {without_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker: {with_reranker_avg_correctness:.4f}")


Average Correctness without Reranker: 0.7908
Average Correctness with Reranker: 0.8654


### 2.7 Clean up
이 노트북에서 생성된 리소스를 삭제하려면 아래 셀의 주석을 해제하고 실행하세요.

In [31]:
print("===============================Knowledge base==============================")
knowledge_base_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Knowledge base==============================
Deleted data source 4Y9VLCW5UK
======== Knowledge base and all data sources deleted =========
Found bucket reranking-kb-1085248
Deleted all objects in bucket reranking-kb-1085248
Deleted bucket reranking-kb-1085248
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_1085248
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_1085248========
 [{'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_1085248', 'PolicyArn': 'arn:aws:iam::461433424192:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_1085248'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_1085248', 'PolicyArn': 'arn:aws:iam::461433424192:policy/AmazonBedrockS3PolicyForKnowledgeBase_1085248'}, {'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_1085248', 'PolicyArn': 'arn:aws:iam::461433424192:policy/AmazonBedrockOSSPolicyForKnowledgeBase